In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pyspark as spark

#read contents of csv into variables
datacenters = pd.read_csv("../data/datacenters.csv")
demand = pd.read_csv("../data/demand.csv")
selling_prices = pd.read_csv("../data/selling_prices.csv")
servers = pd.read_csv("../data/servers.csv")

In [22]:
import numpy as np
from scipy.optimize import minimize

#for DC1 and servergen cpus1 over first TIMESTEPS timesteps

TIMESTEPS = 72

dc1_cap = datacenters[datacenters["datacenter_id"] == "DC1"]["slots_capacity"].iloc[0]
cpu_energies = servers["energy_consumption"].to_numpy()
purchase_prices = servers["purchase_price"].to_numpy()
capacity = servers["capacity"].to_numpy()

cpus1_energy= servers[servers["server_generation"] == "CPU.S1"]["energy_consumption"].iloc[0]
cpus1_purchasecost = servers[servers["server_generation"] == "CPU.S1"]["purchase_price"].iloc[0]
demand2 = demand[demand["latency_sensitivity"] == "low"]
demands = demand2.drop(columns=["latency_sensitivity","time_step"]).iloc[0:TIMESTEPS].to_numpy()
selling_prices_array = selling_prices[selling_prices["latency_sensitivity"] == "low"]["selling_price"].to_numpy()
maint_prices = servers["average_maintenance_fee"].to_numpy()

timestep_array = np.arange(1,96,1)
cpus1_maintenance_cost = servers[servers["server_generation"] == "CPU.S1"]["average_maintenance_fee"].iloc[0]
ts_array = 1.5 * timestep_array
maintenance_cost_array = np.empty((95,7))
for i in range(7):
    maintenance_cost_array[:,i] = (1+ ts_array/96 * np.log2(ts_array/96)) * maint_prices[i]
epsilon = 0.00000001

#where x is an array containing what servergen was bought at each timestep for all servergens
def capacity_constraint(x):
    x = np.reshape(x,(TIMESTEPS,7))
    total = 0
    #servernumber * slotsize to get slots occupied
    #for cpu
    occupied_slots = np.sum(x[:,0:4] * 2)
    #for gpu
    total = occupied_slots + np.sum(x[:,4:7] * 4)
    # #get total number of servers purchased
    # total = np.sum(x)
    # #servernumber * slotsize to get slots occupied
    # total = total * 2
    #constraint used cap has to be less than dc1_cap
    return dc1_cap - total

#get utilisation over the timesteps
def utilisation(x, y):
    #print(x)
    #array of utilisation of each server at eachtimestep
    util = []
    for i in range(TIMESTEPS):
        #array of bought servergens at this timestep
        servergen = x[i]
        #array of min(servergen_supply,demand) at this timestep for all servergens
        s_d_min = y[i]
        #get cumulative sum of number of servers to get total owned at each timestep
        cumsum = []
        total = epsilon
        #calc number of servers at each timestep and their cap
        s_d_sum = []
        sum = 0
        for j in range(len(servergen)):
            sum += s_d_min[j]
            cumsum.append((total + servergen[j]) * capacity[j])
            total = cumsum[j]
        #cumsum = np.cumsum(servergen)
        #get their capacity
        #get demand met for servergen i
        #get fraction of servergen utilised at each timestep
        util.append(sum)

    return util, cumsum

def lifespan(x,y):
    #get number of servers bought for timesteps (servergen doesnt matter)
    ts_sum = np.sum(x, axis=1)
    cumsum = np.cumsum(ts_sum)+epsilon
    life_spans = []
    for i in range(1,TIMESTEPS+1):
        multiplication_arr = np.arange(i,0,-1)
        #array[i-1] = np.divide(np.sum(np.multiply(ts_sum[0:i], multiplication_arr[0:i])), 96)
        sum = 0
        for j in range(i):
            m = ts_sum[j] * multiplication_arr[j]
            sum += m
        life_spans.append(sum)

    # ts1x = x[0]
    # ts2x = x[1]
    # ts1ls = (ts1x/96) / (ts1x +epsilon)
    # ts2ls = ((ts1x)/48 + ts2x/96)/ (ts1x+ts2x+epsilon)

    return life_spans, cumsum

def profit(x, y):
    #get cumulative sum of number of servers for all servergens
    revenues = []
    #get generated revenue at each timestep
    for i in range(TIMESTEPS):
        servergen = x[i, :]
        #get demand met
        supply = y[i,:]
        revenue = 0
        for j in range(7):
            revenue += supply[j] * selling_prices_array[j].astype("int") * capacity[j]
        revenues.append(revenue)
    
    #calc energycost for all servergens at dc1
    energy_costs = cpu_energies * 0.25

    timestep_costs = []
    for i in range(len(x)):
        #get servers that have been maintained (not new)
        maintained_servers = x[:i]
        #calc cost of new servers and add to overall cost at end
        new_cost = x[i] * np.rint((purchase_prices + energy_costs + maintenance_cost_array[i])).astype("int")
        new_cost = np.sum(new_cost)
        #calc energy + maintenance cost
        energy_and_maint = maintenance_cost_array[:i] + energy_costs
        energy_and_maint = np.rint(energy_and_maint).astype("int")
        #multiply corresponding servers with their cost to get total for servergen at each ts
        maint_cost = np.sum(np.multiply(maintained_servers, energy_and_maint[:i]))
        # if(maint_cost.size <= 0):
        #     maint_cost = np.zeros((7))
        timestep_costs.append(maint_cost + new_cost)

    profit = []
    for i in range(len(revenues)):
        profit.append(revenues[i]-timestep_costs[i])
    # print(revenues[0])
    # print(len(timestep_costs))
    return profit

    # ts1x = x[0]
    # ts2x = x[1]
    # ts1revenue = min(60*ts1x, 4000) * 10
    # ts2revenue = min(60*ts1x+60*ts2x, 8160) * 10

    # maint_cost = maintenance_cost(x)
    # ts1cost = (1500 + energycost + maint_cost[0]) * ts1x
    # ts2cost = (energycost + maint_cost[1]) * ts1x + (energycost + maint_cost[0]) * ts2x

    # ts1p = ts1revenue-ts1cost
    # ts2p = ts2revenue-ts2cost
    # return [ts1p, ts2p]

def maintenance_cost(x):
    return maintenance_cost_array[0:len(x)]
    # ts1x = x[0]
    # ts2x = x[1]

    # ts1maintenance = (1+ 1.5/96 * np.log2(1.5/96)) * cpus1_maintenance_cost
    # ts2maintenance = (1+ 3/96 * np.log2(3/96)) * cpus1_maintenance_cost
    # return [ts1maintenance, ts2maintenance]


def objective_func(x, y):
    x = np.reshape(x,(TIMESTEPS, 7))
    y = np.reshape(y,(TIMESTEPS,7))
    P = profit(x, y)
    Objective = np.sum(P)
    return Objective

In [23]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp

def max_profit():
    # Create the solver
    solver = pywraplp.Solver.CreateSolver("SAT")

    # Variables
    # x is the bought servergens at each timestep
    x = []
    # y is the min(supply, demand) at each timestep for each server
    y = []
    c = 0
    for i in range(TIMESTEPS):
        #generate cpu servers
        for j in range(4):
            x.append(solver.IntVar(0, int(dc1_cap/2), f'x{c}'))
            y.append(solver.IntVar(0, int(dc1_cap/2), f'y{c}'))
            c+=1
        #generate gpu servers
        for j in range(3):
            x.append(solver.IntVar(0, int(dc1_cap/4), f'x{c}'))
            y.append(solver.IntVar(0, int(dc1_cap/4), f'y{c}'))
            c+=1
    # x = [solver.IntVar(0, int(dc1_cap/4), f'x{i}') for i in range(TIMESTEPS*7)]
    # y = [solver.IntVar(0, int(dc1_cap/4), f'y{i}') for i in range(TIMESTEPS*7)]
    # z is the accumulated number of servers at each timestep
    #z = [solver.IntVar(0, int(dc1_cap/2), f'z{i}') for i in range(TIMESTEPS*7)]
    print("Number of variables =", solver.NumVariables())

    # Constraints
    #solver.Add(sum(weights[i] * x[i] for i in range(num_items)) <= capacity)
    for i in range(7):
        cumsum = 0
        counter = i
        for i in range(TIMESTEPS):
            cumsum += x[counter]
            counter = counter+7

    print(capacity_constraint(x))
    #solver.Add(capacity_constraint(x) >= 0)
    for i in range(7):
        total_x = 0
        counter = i
        for j in range(TIMESTEPS):
            total_x += x[counter]
            if(i < 4):
                solver.Add(total_x*2 <= dc1_cap)
            else:
                solver.Add(total_x*4 <= dc1_cap)
            solver.Add(y[counter] <= demands[j][i])
            solver.Add(y[counter] <= total_x*60)
            counter=counter+7
    #print("Number of constraints =", solver.NumConstraints())

    # decision_builder = solver.Phase(
    # x, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

    # Objective
    solver.Maximize(objective_func(x, y))

    # Solve
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Total value =', solver.Objective().Value())
        for i in range(7):
            print(f'Item {i}: {x[i].solution_value()}')
    else:
        print('The problem does not have an optimal solution.')

if __name__ == '__main__':
    max_profit()

Number of variables = 1008
(-(((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((2 * x0) + (2 * x1)) + (2 * x2)) + (2 * x3)) + (2 * x7)) + (2 * x8)) + (2 * x9)) + (2 * x10)) + (2 * x14)) + (2 * x15)) + (2 * x16)) + (2 * x17)) + (2 * x21)) + (2 * x22)) + (2 * x23)) + (2 * x24)) + (2 * x28)) + (2 * x29)) + (2 * x30)) + (2 * x31)) + (2 * x35)) + (2 * x36)) + (2 * x37)) + (2 * x38)) + (2 * x42)) + (2 * x43)) + (2 * x44)) + (2 * x45)) + (2 * x49)) + (2 * x50)) + (2 * x51)) + (2 * x52)) + (2 * x56)) + (2 * x57)) + (2 * x58)) + (2 * x59)) + (2 * x63)) + (2 * x64)) + (2 * x65)) + (2 * x66)) + (2 * x70)) + (2 * x71)) + (2 * x72)) + (2 * x73)) + (2 * x77)) + (2 * x78)) + (2 * x79)) + (2 * x80)) + (2 * x84)) + (2 * x85)) + (2 * x86)) + (2 * x87)) + (2 * x91)) + (2